In [5]:
import os
os.chdir('./../../')
from src.basic_utils import *
from src.pivot_utils import *

In [ ]:
data_path = 'data/'

train = pd.read_parquet(data_path+'train(song_trim).parquet')
# val = pd.read_json(data_path+'val(lower).json')
# test = pd.read_json(data_path+'test(lower).json')
# song_meta = load_json(data_path+'song_meta.json')
# genre_gn_all = pd.read_json(data_path+'genre_gn_all.json', typ = 'series')

### make_file.ipynb 내에서 만들어진 tag_song_dict / song_tag_dict 불러오기
# tag_song_dict = load_json(data_path+'tag_song_dict.json')
# song_tag_dict = load_json(data_path+'song_tag_dict.json')

In [6]:
train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))

In [10]:
year_thres = [([i for i in range(0,11)], 1),
              ([i for i in range(11,14)], 1),
              ([14], 1), ([15], 1), ([16], 2), ([17], 2),
              ([18], 4), ([19], 5), ([20], 2)]

In [11]:
for small_years, thres in year_thres:
    make_pv_file(train, small_years, thres)

train only year (6453, 7)
train best playlist (5797, 7)
before  (36797, 1)
after  (21630, 1)


100%|██████████| 5797/5797 [11:39<00:00,  8.29it/s]


(118456, 3)
spend time  11.9
complete train
최종 shape :  (21630, 5719)
train only year (6531, 7)
train best playlist (6282, 7)


  0%|          | 0/6282 [00:00<?, ?it/s]

before  (53654, 1)
after  (31659, 1)


...

100%|██████████| 6604/6604 [31:09<00:00,  3.53it/s]


(245507, 3)
spend time  31.67
complete train
최종 shape :  (35028, 6500)
train only year (13208, 7)
train best playlist (11853, 7)


  0%|          | 3/11853 [00:00<07:08, 27.65it/s]

before  (82760, 1)
after  (50283, 1)


 89%|████████▊ | 10498/11853 [58:32<05:44,  3.93it/s]